# Publish Regular 2D Grid from Image

This notebook shows how you can sign in and publish a Regular 2D Grid geoscience object from an image file (JPEG, PNG, TIFF, BMP, GIF, etc.) to your chosen Evo workspace.

**Important:** This notebook requires Python 3.10, 3.11, or 3.12 (not 3.14+) due to evo.notebooks dependencies.

In the first cell we create a ServiceManagerWidget which will open a browser window and ask you to sign in.

Once signed in, a widget will be displayed to allow you to select your organisation and an Evo workspace.

__Required:__ In Cell 2, replace `"your-client-id"` with your Evo app client ID before running the cell.

In [ ]:
from evo.notebooks import ServiceManagerWidget

manager = await ServiceManagerWidget.with_auth_code(client_id="your-client-id").login()

## Select Organization, Hub, and Workspace

This cell prepares your active Evo context for publishing.

It will:
- List available organizations and select one (by `org_index`)
- List hubs and select one (by `hub_index`)
- List workspaces and select one (by `workspace_index`)
- Create a new workspace only if none exist

Before running, update the index values to match the organization, hub, and workspace you want to use.

In [ ]:
# Select organization and workspace, or create one if needed
service_mgr = manager._service_manager

# Variables to store workspace info
new_workspace = None
selected_workspace_id = None

# List available organizations
orgs = list(service_mgr.list_organizations())
print("Available Organizations:")
for i, org in enumerate(orgs):
    print(f"  {i}: {org.display_name} (ID: {org.id})")

if not orgs:
    print("  No organizations found!")
else:
    # Select your organization by index (change the number if needed)
    org_index = 0  # Change this to the index of your organization
    service_mgr.set_current_organization(orgs[org_index].id)
    print(f"\n✅ Selected organization: {orgs[org_index].display_name}")

    # List available hubs
    hubs = list(service_mgr.list_hubs())
    print("\nAvailable Hubs:")
    for i, hub in enumerate(hubs):
        print(f"  {i}: {hub.display_name} (Code: {hub.code})")

    if hubs:
        # Select your hub by index (change the number if needed)
        hub_index = 0  # Change this: 0=Australia East, 1=Canada Central, 2=South Africa North and so on...
        service_mgr.set_current_hub(hubs[hub_index].code)
        print(f"\n✅ Selected hub: {hubs[hub_index].display_name}")

        # List available workspaces
        workspaces = list(service_mgr.list_workspaces())
        print("\nAvailable Workspaces:")

        if workspaces:
            for i, ws in enumerate(workspaces):
                print(f"  {i}: {ws.display_name} (ID: {ws.id})")

            # Select your workspace by index (change the number if needed)
            workspace_index = 0  # Change this to the index of your workspace
            service_mgr.set_current_workspace(workspaces[workspace_index].id)
            selected_workspace_id = workspaces[workspace_index].id
            print(f"\n✅ Selected workspace: {workspaces[workspace_index].display_name}")
        else:
            print("  No workspaces found!")
            print("\n💡 Creating a new workspace...")

            # Create a workspace
            from evo.workspaces import WorkspaceAPIClient
            from datetime import datetime

            connector = manager.get_connector()
            workspace_client = WorkspaceAPIClient(connector, orgs[org_index].id)

            # Create workspace with timestamp
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            workspace_name = f"Image Grid Workspace {timestamp}"

            try:
                new_workspace = await workspace_client.create_workspace(
                    name=workspace_name, description="Workspace for Image to Grid conversion"
                )

                selected_workspace_id = new_workspace.id
                print(f"✅ Workspace created: {new_workspace.display_name}")
                print(f"   Workspace ID: {new_workspace.id}")

                from evo.service_manager.manager import _State

                current_workspaces = list(service_mgr.list_workspaces())
                current_workspaces.append(new_workspace)

                service_mgr._ServiceManager__state = _State(
                    organizations=list(service_mgr.list_organizations()),
                    workspaces=current_workspaces,
                    selected_org_id=orgs[org_index].id,
                    selected_hub_code=hubs[hub_index].code,
                    selected_workspace_id=new_workspace.id,
                )

                print("✅ Workspace selected and ready for publishing!")

            except Exception as e:
                print(f"❌ Failed to create workspace: {e}")
                import traceback

                traceback.print_exc()
    else:
        print("  No hubs found!")

## Convert and Publish Image to Regular 2D Grid

In the cell below we:
1. Specify the path to our image file (JPEG, PNG, TIFF, etc.)
2. Set the grid parameters (origin, cell_size, coordinate reference system)
3. Add optional tags
4. Call `convert_image_to_grid` to convert and publish the image

The image will be:
- Converted to grayscale
- Pixel values extracted and stored in a parquet file
- Formatted according to the regular-2d-grid schema
- Published to your Evo workspace

**Note:** If you don't have workspaces available, use `evo_workspace_metadata` parameter instead of `service_manager_widget`.

In [ ]:
# Install local package into this notebook kernel (run once per environment)
%pip install -e ../..

In [ ]:
from evo.data_converters.image_to_2dgrid import convert_image_to_grid
from pathlib import Path

# Path to your image file (JPEG, PNG, TIFF, etc.)
# Using relative path from notebook location
image_file = "data/input/sample_gradient.jpg"

# Grid origin in world coordinates [x, y, z]
origin = [572565.0, 6839415.0, 1000.0]

# Cell size [cell_size_x, cell_size_y]
cell_size = [30.0, 30.0]

# Coordinate Reference System (optional)
crs = {"epsg_code": 32632}  # WGS 84 / UTM zone 32N

# Tags to add to the geoscience object
tags = {"Source": "Jupyter Notebook", "Type": "Image Grid"}

# Name for the grid (optional, defaults to filename)
grid_name = f"Image Grid - {Path(image_file).stem}"

# Description (optional)
description = "A 2D grid generated from image file"

# Upload path in Evo workspace
upload_path = "grids/image_imports"

print("Converting and publishing image to Evo workspace...")
print(f"Image: {Path(image_file).name}")
print(f"Workspace: {selected_workspace_id}")
print(f"Origin: {origin}")
print(f"Cell size: {cell_size}")

# Convert and publish
results = convert_image_to_grid(
    image_path=image_file,
    origin=origin,
    cell_size=cell_size,
    coordinate_reference_system=crs,
    tags=tags,
    name=grid_name,
    description=description,
    service_manager_widget=manager,
    upload_path=upload_path,
    publish_objects=True,
    overwrite_existing_objects=True,
)

# Print results
print("\n✅ Successfully published!")
print("\nPublished objects:")
for obj_metadata in results:
    print(f"  - {obj_metadata.name}")
    print(f"    ID: {obj_metadata.id}")

## Simple Example with Defaults

If you don't need custom parameters, you can use a simpler call with default values:

In [ ]:
from evo.data_converters.image_to_2dgrid import convert_image_to_grid

# Minimal example with defaults:
# - origin: [0, 0, 0]
# - cell_size: [1, 1]
# - no CRS
# - name derived from filename

results = convert_image_to_grid(
    image_path="data/input/sample_gradient.jpg",
    service_manager_widget=manager,
    upload_path="grids",
    tags={"Source": "Jupyter Notebook"},
)

print(f"Published: {results[0].name}")

## Convert Without Publishing (for testing)

You can also convert the JPEG without publishing to inspect the resulting object:

In [ ]:
from evo.data_converters.image_to_2dgrid import convert_image_to_grid
import json

# Convert but don't publish
grid_objects = convert_image_to_grid(
    image_path="data/input/sample_gradient.jpg",
    service_manager_widget=manager,
    publish_objects=False,  # Don't publish
)

# Inspect the grid object
grid = grid_objects[0]
print(f"Grid name: {grid.name}")
print(f"Grid size: {grid.size}")
print(f"Cell size: {grid.cell_size}")
print(f"Origin: {grid.origin}")
print(f"Number of cell attributes: {len(grid.cell_attributes)}")

grid_dict = grid.as_dict()  # Use as_dict() instead of to_dict()
print("\nJSON representation:")
print(json.dumps(grid_dict, indent=2))